# Student Loan Calculator (WIP)

I wanted to build something useful whilst learning more about pandas and ipy widgets... 

Is it worth repaying a student loan faster than necessary??

#### INSTRUCTIONS:
-Click Cell -> Run all 

-Use sliders to change variables and see what combinations work well. The main points of interest would probably be starting salary, loan value, overpay value and overpay duration.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
import matplotlib

#%matplotlib notebook

In [ ]:
df = pd.DataFrame(columns=['Year', 'Salary', 'Tax', 'Outstanding_Loan', 'Forced Repayment', 'Voluntary Repayment', 'Net Salary', 'Loan Interest', 'Plan 2 Loan', 'Plan 2 Forced', 'Plan 2 Net'])
df.Year = range(30)


In [13]:
### one big function to recalculate everything when parameters are updated

def calculate_values(initial_salary, initial_loan, inflation, tax_free_allowance, low_tax_rate, low_tax_band, mid_tax_rate, 
                     mid_tax_band, high_tax_rate, repay_rate, repay_thresh, payrise, rise_interval, voluntary_value, 
                     voluntary_duration):
    
    yearly_salary = np.zeros(len(df))
    yearly_salary[0] = initial_salary
    for i in range(len(yearly_salary)-1):
        yearly_salary[i+1] = yearly_salary[i] * (1+inflation)

        ## come back and make this salary boost customizeable
        if i % rise_interval == 0 and i !=0:
            yearly_salary[i+1] += payrise

    df.Salary = yearly_salary

    ## should really fix this to take account of all three tax bands
    df.Tax = np.where(df['Salary'] >= 50000, (df.Salary - tax_free_allowance) * 0.4, (df.Salary - tax_free_allowance) * 0.2)
    df.loc[0, 'Outstanding_Loan'] = initial_loan
    df.loc[0, 'Plan 2 Loan'] = initial_loan

    # setup how much we want to repay voluntarily
    voluntary = np.zeros(len(df))
    for i in range(voluntary_duration):
        voluntary[i] = voluntary_value
    df['Voluntary Repayment'] = voluntary

    forced = np.ones(len(df)) * 1000
    paidoff = False
    plan2paidoff = False
    plan2forced = np.ones(len(df)) * 1000
    
    # setup a loop to populate the dataframe with dependent values
    for i in range(len(df)):
        
        
        # sort out how much interest is charged each year
        if df.loc[i, 'Salary'] < 26575:
            df.loc[i, 'Loan Interest'] = inflation + 1
        elif df.loc[i, 'Salary'] < 47835:
            df.loc[i, 'Loan Interest'] = 1 + inflation +  (df.loc[i, 'Salary'] - 26575) * 0.03 / 21260
        else:
            df.loc[i, 'Loan Interest'] = inflation + 1.03


        # calculate the mandatory repayment amount if it's non negative

        if (df.loc[i, 'Salary'] > repay_thresh) and not paidoff:
            forced[i] = (df.loc[i, 'Salary'] - repay_thresh) * repay_rate
            df['Forced Repayment'] = forced
        else:
            df.loc[i, 'Forced Repayment'] = 0
            
        if (df.loc[i, 'Salary'] > repay_thresh) and not plan2paidoff:
            plan2forced[i] = (df.loc[i, 'Salary'] - repay_thresh) * repay_rate
            df['Plan 2 Forced'] = plan2forced
        else:
            df.loc[i, 'Plan 2 Forced'] = 0

        # multiply loan by interest rate
        df.loc[i, 'Outstanding_Loan'] *= df['Loan Interest'][i]
        df.loc[i, 'Plan 2 Loan'] *= df['Loan Interest'][i]

        # reduce debt by amount repaid
        if not paidoff:
            if df.loc[i, 'Outstanding_Loan'] - df['Forced Repayment'][i] - df['Voluntary Repayment'][i] > 0:
                df.loc[i+1, 'Outstanding_Loan'] = df.loc[i, 'Outstanding_Loan'] - df['Forced Repayment'][i] - df['Voluntary Repayment'][i]
            else:
                df.loc[i, 'Forced Repayment'] = df.loc[i, 'Outstanding_Loan'] - df['Voluntary Repayment'][i]
                paidoff = True
                df.loc[i+1, 'Outstanding_Loan'] = 0
        else:
            df.loc[i, 'Outstanding_Loan'] = 0
            
        if not plan2paidoff:
            if df.loc[i, 'Plan 2 Loan'] - df['Plan 2 Forced'][i] > 0:
                df.loc[i+1, 'Plan 2 Loan'] = df.loc[i, 'Plan 2 Loan'] - df['Plan 2 Forced'][i]
            else:
                df.loc[i, 'Plan 2 Forced'] = df.loc[i, 'Plan 2 Loan']
                plan2paidoff = True
                df.loc[i+1, 'Plan 2 Loan'] = 0
        else:
            df.loc[i, 'Plan 2 Loan'] = 0

        # calculate net salary for the current year by subtracting tax and repayments
        df.loc[i, 'Net Salary'] = df.loc[i, 'Salary'] - df['Tax'][i] - df['Forced Repayment'][i] - df['Voluntary Repayment'][i]
        df.loc[i, 'Plan 2 Net'] = df.loc[i, 'Salary'] - df['Tax'][i] - df['Plan 2 Forced'][i]
        
    # calculate lifetime repayments
    
    plan1_tot = df['Forced Repayment'].sum() + df['Voluntary Repayment'].sum()
    plan2_tot = df['Plan 2 Forced'].sum()
    if plan1_tot < plan2_tot:
        bar_colour = 'lawngreen'
    elif plan1_tot == plan2_tot:
        bar_colour = 'darkgrey'
    else:
        bar_colour = 'red'
        
    fig = plt.figure(figsize=[16,9])
    gs = matplotlib.gridspec.GridSpec(1, 2, width_ratios=[3, 1], wspace=0) 
    ax0 = plt.subplot(gs[0])
    ax0.plot(df.Year, df['Plan 2 Loan'], color='blue', label='No Repayments Debt')
    ax0.plot(df.Year, df['Plan 2 Net'], color='blue', ls='--', label='No Repayments Net Salary')
    ax0.plot(df.Year, df['Outstanding_Loan'], color='orange', label='Extra Repayments Debt')
    ax0.plot(df.Year, df['Net Salary'], color='orange', ls='--', label='Extra Repayments Net Salary')
    plt.ylabel('Loan/Salary (£)')
    plt.xlabel('Years from graduation')
    plt.legend()
    ax1 = plt.subplot(gs[1])
    ax1.tick_params(axis='y', left=False, right=True, labelleft=False, labelright=True)
    ax1.bar(['With Repayments', 'Without'] , [plan1_tot,plan2_tot], color=bar_colour)
    plt.ylabel('Total Lifetime Cost')
    #plt.legend()
    plt.show()

In [14]:
initial_salary = widgets.IntSlider(value=36000, min=0, max=100000, step=50, continuous_update=False, description='Initial Salary')
initial_loan = widgets.FloatSlider(value=43186.95,min=0.0, max=100000, step=0.5, continuous_update=False, description='Loan Value')
inflation = widgets.FloatSlider(value=0.0202,min=0.0,max=0.15,step=0.002,continuous_update=False, description='Inflation Rate')
tax_free_allowance = widgets.IntSlider(value=12500,min=0,max=50000,step=50,continuous_update=False, description='No Tax Thrsh')
low_tax_rate = widgets.FloatSlider(value=0.2,min=0.0,max=1.0,step=0.01,continuous_update=False, description='Lo Tax Rate')
low_tax_band = widgets.IntSlider(value=50000, min=0, max=200000, step=50, continuous_update=False, description='Lo Tax Thrsh')
mid_tax_rate = widgets.FloatSlider(value=0.4,min=0.0,max=1.0,step=0.01,continuous_update=False, description='Md Tax Rate')
mid_tax_band = widgets.IntSlider(value=150000, min=0, max=200000, step=50, continuous_update=False, description='Md Tax Thrsh')
high_tax_rate = widgets.FloatSlider(value=0.45,min=0.0,max=1.0,step=0.01,continuous_update=False, description='Hi Tax Rate')
repay_rate = widgets.FloatSlider(value=0.09,min=0.0,max=1.0,step=0.01,continuous_update=False, description='Repay Rate')
repay_thresh = widgets.IntSlider(value=26568,min=0, max=50000, step=1, continuous_update=False, description='Repay Thrsh')
payrise = widgets.IntSlider(value=3000, min=0, max=15000, step=5, continuous_update=False, description='Payrise Val')
rise_interval = widgets.IntSlider(value=5, min=1, max=30, step=1, continuous_update=False, description='Rise Interval')
voluntary_value = widgets.IntSlider(value=0, min=0, max=50000, step=5, continuous_update=False, description='Overpay Val')
voluntary_duration = widgets.IntSlider(value=5, min=1, max=30, step=1, continuous_update=False, description='Overpay Dur')

lvl1 = widgets.HBox([initial_salary, initial_loan, repay_rate])
lvl2 = widgets.HBox([inflation, tax_free_allowance, repay_thresh])
lvl3 = widgets.HBox([low_tax_rate, low_tax_band, voluntary_value])
lvl4 = widgets.HBox([mid_tax_rate, mid_tax_band, voluntary_duration])
lvl5 = widgets.HBox([high_tax_rate, payrise, rise_interval])
                                       
ui = widgets.VBox([lvl1, lvl2, lvl3, lvl4, lvl5])
                                       
out = widgets.interactive_output(calculate_values, {'initial_salary':initial_salary, 'initial_loan':initial_loan, 
        'inflation':inflation, 'tax_free_allowance':tax_free_allowance, 'low_tax_rate':low_tax_rate, 'low_tax_band':low_tax_band,
        'mid_tax_rate':mid_tax_rate, 'mid_tax_band':mid_tax_band, 'high_tax_rate':high_tax_rate, 'repay_rate':repay_rate, 
        'repay_thresh':repay_thresh, 'payrise':payrise, 'rise_interval':rise_interval, 'voluntary_value':voluntary_value, 
                                                    'voluntary_duration':voluntary_duration})
                                       
display(ui, out)

Output()

In [16]:
## This cheeky bit of formatting hides all the code which is niiiiceee

from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
div.input {
    display:none;
}
</style>
""")